In [65]:
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from scipy import linalg
from scipy.stats import mstats
from scipy.optimize import minimize
from scipy.optimize import brentq
import matplotlib.pyplot as plt
import seaborn as sns
import pdb
import copy
from sklearn import datasets as ds
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
%matplotlib inline

In [12]:
#!pip install pandas
#!pip install plotly
#!pip install scipy
#!pip install matplotlib
#!pip install seaborn
#!pip install sklearn

    100% |████████████████████████████████| 12.4MB 2.2MB/s ta 0:00:011
  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /home/slava/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [54]:
plotly.tools.set_credentials_file(username='slava-mazur', api_key='0Up94ov3B6l4c11Pbxrc')

In [22]:
my_seed = 777

In [14]:
from distutils.sysconfig import get_python_lib
print(get_python_lib())

/home/slava/anaconda3/envs/anomaly/lib/python3.6/site-packages


In [19]:
X, y = ds.load_iris(return_X_y=True)
X[0:5,:]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [23]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.33, random_state = my_seed)
X_train.shape

(100, 4)

In [136]:
# Create a plotly trace
def plotly_scatter3d(data, Class = None, text = None, show_text = False, marker_size = 3, colors = ['gray', 'blue', 'red']):
    if (text is None) or not show_text:
        mode = 'markers'  
    else: 
        mode = 'markers+text'
    traces = list()
    if Class is None:
        trace = go.Scatter3d(
            x = data[:,0],
            y = data[:,1],
            z = data[:,2],
            mode = mode,
            marker = dict(size=marker_size, color = colors[0]),
            text = text
        )
        traces.append(trace)
    else:
        classes = set(Class)
        clr = 0
        for c in classes:
            trace = go.Scatter3d(
                x = data[Class == c,0],
                y = data[Class == c,1],
                z = data[Class == c,2],
                mode = mode,
                marker = dict(size=marker_size, color = colors[clr]),
                text = text
            )
            traces.append(trace)
            clr += 1
            
    
    layout = go.Layout(showlegend=(Class is not None))
    fig = go.Figure(data=traces, layout=layout)

    return py.iplot(fig, filename='3d-scatter')

In [126]:
# some variety of kernel functions
def gauss_kernel_norm(x):
    vars_ = [np.var(x[:, j]) for j in range(x.shape[1])]
    
    def calc(y, z):
        tmp = y - z
        return np.exp(-np.mean(tmp * tmp / vars_))
    
    return calc

def mahalanobis_kernel_norm(x):
    cov_ =  np.linalg.inv(np.cov(x, rowvar=False))
    
    def calc(y, z):
        tmp = y - z
        d = np.dot(np.dot(tmp.T, cov_), tmp)
        return np.exp(-d / len(y))
    
    return calc

def dot_kernel_norm(y, z):
    cv = np.inner(y, z)
    ny = np.inner(y, y)
    nz = np.inner(z, z)
    return cv / (np.sqrt(ny) * np.sqrt(nz))

def dot_kernel(y, z):
    cv = np.inner(y, z)
    return cv / len(y)



In [27]:
# Kernel (Gram) matrix calculation and centering
def kern_matrix(data, fKernel):
    K = np.zeros([data.shape[0], data.shape[0]]) 
    for i in range(data.shape[0]): 
        for j in range(i, data.shape[0]):
            K[i, j] = K[j, i] = fKernel(data[i,:], data[j, :])
    return K

def kern_matrix_test(X, Y, fKernel):
    K = np.zeros([X.shape[0], Y.shape[0]]) 
    for i in range(X.shape[0]): 
        for j in range(Y.shape[0]):
            K[i, j] = fKernel(X[i,:], Y[j, :])
    return K

def kern_matrix_centered(data, basis_events, u, fKernel):
    K0 = kern_matrix(basis_events, fKernel)
    sq_norm = np.dot(u.T, np.dot(K0, u))
    K = kern_matrix(data, fKernel)
    Y = np.zeros([data.shape[0], basis_events.shape[0]])
    for i in range(data.shape[0]): 
        for j in range(basis_events.shape[0]):
            Y[i, j] = fKernel(data[i,:], basis_events[j, :])
    y = np.dot(Y, np.reshape(u, [basis_events.shape[0], 1]))
    
    for j in range(K.shape[1]): 
        K[:, [j]] -= y
    y = y.T
    for i in range(K.shape[0]): 
        K[[i], :] -= y
    return K + sq_norm

def kern_matrix_centered_test(X, Y, basis_events, u, fKernel):
    K0 = kern_matrix(basis_events, fKernel)
    sq_norm = np.dot(u.T, np.dot(K0, u))
    K = kern_matrix_test(X, Y, fKernel)
    Z = np.zeros([X.shape[0], basis_events.shape[0]])
    for i in range(X.shape[0]): 
        for j in range(basis_events.shape[0]):
            Z[i, j] = fKernel(X[i,:], basis_events[j, :])
    y = np.dot(Z, np.reshape(u, [basis_events.shape[0], 1]))
    
    for j in range(K.shape[1]): 
        K[:, [j]] -= y
        
    Z = np.zeros([Y.shape[0], basis_events.shape[0]])
    for i in range(Y.shape[0]): 
        for j in range(basis_events.shape[0]):
            Z[i, j] = fKernel(Y[i,:], basis_events[j, :])
    y = np.dot(Z, np.reshape(u, [basis_events.shape[0], 1]))
    y = y.T
    for i in range(K.shape[0]): 
        K[[i], :] -= y
        
    return K + sq_norm


In [98]:
# Distance-to-center related calculations
def calc_rmin2(K):
    k = (K - np.identity(K.shape[0])).flatten().argmax()
    i = k // K.shape[0]
    j = k % K.shape[0]
    return K[i, i] + K[j, j] - 2*K[i, j]

def square_dists_to_center(K, w):
    w2 = np.array(w * w)
    W2 = np.sum(w2)
    w2 = w2 / W2
    return K.diagonal() - 2 * np.dot(K, w2) + np.dot(np.dot(w2.transpose(), K), w2)

def init_approx2(K, rmin2):
    w = K.sum(axis=0) 
    r2 = square_dists_to_center(K, w)
    a = np.sqrt(r2.max() * r2.min())
    w = a / (a + r2)
    r = np.sqrt(rmin2 / 2);
    if (r > 0):
        amax = r
        wmax = amax / (amax + r*r)
    else: 
        wmax = 1
    
    return np.minimum(w, wmax-1e-6)

def scale_factor(K, w):
    R = square_dists_to_center(K, w)
    return np.sqrt(np.prod([R.min(), R.max()]))

def scale_factor_reduced(K, w, rmin2):
    R = square_dists_to_center(K, w)
    R = [R.min(), R.max()]
    rmin = [rmin2/2, rmin2/2]
    return np.sqrt(np.prod(np.maximum(R, rmin)))

def func_j2(w, K, debug_print = False):
    #pdb.set_trace()
    a = scale_factor(K, w)
    w2 = np.reshape(w * w, [len(w), 1])
    W2 = np.sum(w2)
    res = 0.5 * (np.sum(K.diagonal() * w2) - np.dot(np.dot(w2.transpose(), K), w2) / W2 + a * np.sum((1 - w)**2)) / K.shape[0]
    if debug_print:
        print("a = {}, wmin = {}, wmax = {}, value = {}".format(a, w.min(), w.max(), res))
    return res.flatten()[0]

def func_j2_reduced(w, K, rmin2, debug_print = False):
    #pdb.set_trace()
    a = scale_factor_reduced(K, w, rmin2)
    w2 = np.reshape(w * w, [len(w), 1])
    W2 = np.sum(w2)
    res = 0.5 * (np.sum(K.diagonal() * w2) - np.dot(np.dot(w2.transpose(), K), w2) / W2 + a * np.sum((1 - w)**2)) / K.shape[0]
    if debug_print:
        print("a = {}, wmin = {}, wmax = {}, value = {}".format(a, w.min(), w.max(), res))
    return res.flatten()[0]

def func_j2_pseudo_grad(w, K):
    a = scale_factor(K, w)
    w2 = w ** 2
    W2 = sum(w2)
    res = np.dot(K.diagonal(), w) - (2 / W2) * np.dot(w.transpose() ** 3, K) + np.dot(w2.transpose(), np.dot(K, w2)) / (W2 * W2) - a * (1 - w)
    return (res / K.shape[0]).flatten()

def func_j2_reduced_pseudo_grad(w, K, rmin2):
    a = scale_factor_reduced(K, w, rmin2)
    w2 = w ** 2
    W2 = sum(w2)
    res = np.dot(K.diagonal(), w) - (2 / W2) * np.dot(w.transpose() ** 3, K) + np.dot(w2.transpose(), np.dot(K, w2)) / (W2 * W2) - a * (1 - w)
    return (res / K.shape[0]).flatten()




In [97]:
def calc_pseudo_opt_weights(K):
    rmin2 = calc_rmin2(K)
    w = init_approx2(K, rmin2)
    ww = w * w
    ww = ww / sum(np.abs(ww))
    o = minimize(fun=func_j2, x0=ww, args=(K), method='CG', jac=func_j2_pseudo_grad, 
             options={'disp':True})
    ww = o['x'] ** 2
    ww = ww / sum(np.abs(ww))
    return ww

In [112]:
# Sugeno measure and integral related calculations
def Frank0(y, Fn, K, X):
    return sum(np.maximum(Fn[X], K[y, X])) / len(X)

def SugenoMeasure(w, lmbda):
    res = np.repeat(0.0, len(w)+1)
    for i in range(len(w)):
        res[i+1] = res[i] + w[i] + lmbda * res[i] * w[i]
    return res[-1]

def SugenoMeasureCum(w, lmbda):
    res = np.repeat(0.0, len(w)+1);
    for i in range(len(w)):
        res[i+1] = res[i] + w[i] + lmbda * res[i] * w[i]
    return res[range(1, len(res))]

def SugenoInt(K, Frank, Fn, w, X, lmbda = None):
    bx = np.array([Frank(y, Fn, K, X) for y in X], dtype=np.float)
    ibx = (-bx).argsort()
    jbx = X[ibx]
    bx = bx[ibx]
    w = w[jbx]
    if (lmbda is None) and (len(w) > 1):
        def f(x):
            return SugenoMeasure(w, x) - 1
        
        m = SugenoMeasure(w, 0)
        if (m > 1):
            lmbda = brentq(f, -1, 0, maxiter=100, full_output=False, disp=False)
        elif (m < 1):
            if (m < 1e-6): # measure of X is practically zero -- need to stop 
                return []
            
            rb = 1
            while m < 1:
                m = SugenoMeasure(w, rb)
                if m < 1:
                    rb = 2 * rb 
            lmbda = brentq(f, 0, rb, maxiter=100, full_output=False, disp=False)
        else: 
            lmbda = 0
            
        m = SugenoMeasureCum(w, lmbda)
    else:
        if (lmbda is None): 
            lmbda = 0
        m = SugenoMeasureCum(w, lmbda)
        m = m / m[-1]
    
    res = np.minimum(bx, m)
    return res[np.argsort(ibx)]

def BasisEvents(K, Frank, w, lmbda = None, eps = 1e-4, min_events = 3, control_singularity = True) :
    # pdb.set_trace()
    res = []
    N = len(w)
    X = np.array(range(N), dtype=np.integer)
    Fn = np.repeat(0.0, N)
    sMax = 0.0
    w0 = copy.deepcopy(w)
    for i in range(N):
        S = SugenoInt(K, Frank, Fn, w0, X, lmbda)
        if (S is None) or (len(S) == 0): 
            break
        Fn1 = copy.deepcopy(Fn)
        Fn[X] = S; 
        progress = min(max(abs(Fn[X]-Fn1[X])), max(S) - sMax)
        jmax = np.argmax(S)
        sMax = S[jmax]
        j = X[jmax]
        res.append(j)
        Xj = set(X) - set([j])
        X = np.sort(list(Xj))
        if (X is None) or (len(X) == 0): 
            break
        Fn[j] = 0.0;
        print("step: {}; selected: {}; sMax = {}; progress = {}".format(i, j, sMax, progress))
        w0[j] = 0.0
        if control_singularity and (len(res) > 1):
            Ku = K[res, :][:, res]
            v = linalg.eigh(Ku, eigvals_only = True)
            if (min(v) < 1e-8):
                print("adding event %d leads to a singular matrix, omitting." % j)
                res = res[:-1]
                progress = 1.0
                sMax = 0.0
        if (len(res) >= min_events) and ((progress < eps) or (sMax > 1 - eps)): 
            break
    
    if (progress < 0) and (len(res) > min_events): 
        return res[:-1] 
    else: 
        return res



In [30]:
# Center related calculations
def BasisEventsWeights(K, be, w):
    w2 = w ** 2
    b = np.dot(K[be, :], w2 / np.sum(w2))
    return np.linalg.solve(K[be, :][:, be], b).flatten()

def square_dists_to_center_reduced(K, u, be):
    res = K.diagonal() - 2 * np.dot(K[:, be], u) + np.dot(np.dot(u.transpose(), K[be, :][:, be]), u)
    return res.flatten()

def center_kern_matrix(K, u):
    y = np.dot(K, u)
    sqnorm = np.dot(u.transpose(), y)
    for i in range(K.shape[0]):
        K[i, :] -= y
    for j in range(K.shape[1]):
        K[:, j] -= y
    return K + sqnorm


In [103]:
# Non-linear eigenvector decomposition
def nl_eigen_vectors(K, L, normalize = False):
    N = K.shape[0]
    w, v = linalg.eigh(K, eigvals = (N-L, N-1))
    if (normalize):
        for i in range(v.shape[0]):
            v[i, :] /= np.sqrt(w) 
    return v[:,::-1]

In [104]:
# Membership and non-linear PCA related calculations
def predict_memberships(x, basis_events, fKernel, u, a, L = 3, normalize = True):
    # pdb.set_trace()
    res = np.zeros([x.shape[0], L+1])
    K0 = kern_matrix_centered(x, basis_events, u, fKernel)
    res[:,0] = a / (a + K0.diagonal())
    if L > 0:
        U = nl_eigen_vectors(K0, L, normalize=normalize)
        tmp = np.dot(U.T, K0)
        res[:,1:] = tmp.T
    return res 

In [117]:
# putting all together
def preprocess_data(x_train, y_train, x_test, fKernel = dot_kernel_norm, cluster = 0, L = 3, normalize = True):
    #pdb.set_trace()
    M = x_train.shape[1]
    data = x_train[y_train == cluster, :]
    print("Calculating kernel")
    K = kern_matrix(data, fKernel)
    print("Finding pseudo-optimal center coeffs")
    w = calc_pseudo_opt_weights(K)
    print("Finding principal events")
    be = BasisEvents(K, Frank=Frank0, w=w, lmbda = None, eps = 1e-4, min_events = L, control_singularity = True) 
    u = BasisEventsWeights(K, be, np.sqrt(w))
    r2 = square_dists_to_center_reduced(K, u, be)
    a = np.sqrt(np.prod([r2.min(), r2.max()]))
    print("Scale: {}, membership range: {} - {}".format(a, a / (a + r2.max()), a / (a + r2.min())))

    basis_events = data[be, :]
    
    print("Calculating memberships")
    y = predict_memberships(x_train, basis_events, fKernel, u, a, L = L, normalize = normalize)
    y1 = predict_memberships(x_test, basis_events, fKernel, u, a, L = L, normalize = normalize)
    res = {'x_train': np.append(x_train, y, axis = 1), 'x_test': np.append(x_test, y1, axis = 1), 'basis_events': basis_events, 'u':u, 'a':a}
    print("Done")
    
    return res

In [180]:
z = preprocess_data(X_train, y_train, X_test, fKernel = dot_kernel_norm, cluster = 0, L = 5, normalize=True)

Calculating kernel
Finding pseudo-optimal center coeffs
         Current function value: 0.038430
         Iterations: 1
         Function evaluations: 93
         Gradient evaluations: 81
Finding principal events
step: 0; selected: 5; sMax = 0.9911139394246439; progress = 0.9911139394246439
step: 1; selected: 0; sMax = 0.9947892761358357; progress = 0.003675336711191801
step: 2; selected: 29; sMax = 0.996124126704249; progress = 0.001334850568413315
step: 3; selected: 27; sMax = 0.9964460567493392; progress = 0.00032193004509017964
step: 4; selected: 7; sMax = 0.9965636281079187; progress = 0.00011757135857948242
adding event 7 leads to a singular matrix, omitting.
step: 5; selected: 24; sMax = 0.9967853769918401; progress = 0.11616311295884701
adding event 24 leads to a singular matrix, omitting.
step: 6; selected: 6; sMax = 0.9969333805013008; progress = 0.07895165339483767
adding event 6 leads to a singular matrix, omitting.
step: 7; selected: 19; sMax = 0.9974132049294636; progres

In [145]:
Mb = np.append(z['x_train'][:,4], z['x_test'][:,4], axis = 0) 
P = np.append(z['x_train'][:,-3:], z['x_test'][:,-3:], axis = 0)
C = np.append(y_train, y_test)
labels = ["%d: %f" % (C[j] , Mb[j]) for j in range(len(C))]

In [135]:
[[P[:,j].min(), P[:,j].max()] for j in range(3)]

[[-0.5376649519348069, 0.07895259846864665],
 [-0.08225987281822979, 0.09872399478223577],
 [-0.0494783183409568, 0.05482792509288456]]

In [146]:
plotly_scatter3d(data=P, Class = C, text = labels, marker_size = 3)

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~slava-mazur/0 or inside your plot.ly account where it is named '3d-scatter'


In [181]:
#x_train = X_train
#x_test = X_test
#x_train = z['x_train']
#x_test = z['x_test']
x_train = z['x_train'][:,-5:]
x_test = z['x_test'][:,-5:]

In [182]:
gs = model_selection.GridSearchCV(AdaBoostClassifier(random_state = my_seed),
                  param_grid={'n_estimators': range(10, 100, 10), 'learning_rate': np.linspace(0.1, 2, num = 20) },
                  cv=5, n_jobs = 8)
gs.fit(x_train, y_train)


GridSearchCV(cv=5, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=777),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'n_estimators': range(10, 100, 10), 'learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [183]:
gs.best_estimator_

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.1, n_estimators=10, random_state=777)

In [178]:
model = AdaBoostClassifier(n_estimators=gs.best_estimator_.n_estimators, random_state=my_seed, learning_rate=gs.best_estimator_.learning_rate)
model.fit(x_train, y_train)
Z = model.predict(x_train)
pd.crosstab(y_train, Z)

col_0,0,1,2
row_0,,,
0,32,0,0
1,0,36,2
2,0,0,30


In [179]:
Z_test = model.predict(x_test)
pd.crosstab(y_test, Z_test)

col_0,0,1,2
row_0,,,
0,18,0,0
1,0,11,1
2,0,0,20
